In [1]:
!pip install pypdf

In [2]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [3]:
## Embedding
!pip install install sentence_transformers

In [4]:
!pip install llama_index

In [5]:
pip install llama-index-llms-huggingface

Note: you may need to restart the kernel to use updated packages.


In [6]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [7]:
documents = SimpleDirectoryReader("./data").load_data()
documents

[Document(id_='bbe295d6-d17e-407f-aa1a-e82a8be2aaef', embedding=None, metadata={'page_label': '1', 'file_name': 'discharge.pdf', 'file_path': '/home/ubuntu/notebooks/data/discharge.pdf', 'file_type': 'application/pdf', 'file_size': 35328, 'creation_date': '2024-05-01', 'last_modified_date': '2024-05-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Name:  ___                     Unit No:   ___  \n  \nAdmission Date:  ___              Discharge Date:   ___  \n  \nDate of Birth:  ___             Sex:   F  \n  \nService: MEDICINE  \n  \nAllergies:  \nNo Known Allergies / Adverse Drug Reactions  \n  \nAttending: ___  \n  \nChief Complaint:  \nWorsening ABD distension and pain  \n  \nMajor Surgical or Invasive Procedure:  \nParacentesis  \n \n  \nHistory 

In [8]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [9]:
import subprocess
from config import BHANU_HF_TOKEN

# Replace 'your_token_here' with your actual Hugging Face token
token = BHANU_HF_TOKEN
input_text = f"{token}\nY\n"  # Combine token and 'Y' input

# Run the Huggingface CLI login command
process = subprocess.Popen(['huggingface-cli', 'login'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate(input_text.encode())

# Print any output or error if necessary (optional)
print(stdout.decode())
print(stderr.decode())


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Add token as git credential? (Y/n) Token is valid (permission: read).
Cannot authenticate through git-credential as no

In [10]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [11]:
!pip install llama-index-embeddings-huggingface

In [24]:
import torch
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import BitsAndBytesConfig


llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
!pip install llama-index-embeddings-langchain

In [25]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

In [26]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

/tmp/ipykernel_58014/2250285859.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context=ServiceContext.from_defaults(


In [27]:
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f220fbaa4f0>, num_workers=None), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f220fbaa4f0>, id_func=<function default_id_func at 0x7f217a3d8550>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.core.service_context_elements.llama_logger.LlamaLogger object at 0x7f229a561970>, callback_manager=<llama_index

In [28]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [29]:
index

In [30]:
query_engine=index.as_query_engine()

In [31]:
response=query_engine.query("what is the Chief complaint(CC)?")
print(response)

/home/ubuntu/discharge_me/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/ubuntu/discharge_me/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


The Chief complaint (CC) is "Worsening ABD distension and pain".


In [32]:
response=query_engine.query("Generate a Brief Hospital Course section for a discharge summary note.")
print(response)

Sure, I'd be happy to help you with that! Here's a Brief Hospital Course section for a discharge summary note based on the information provided:

Brief Hospital Course:

The patient was admitted to the hospital for ascites due to portal hypertension. Upon admission, the patient was found to have a distended abdomen with mild diffuse tenderness, and there was no palpable liver edge. The patient was started on furosemide 40 mg orally daily, spironolactone 50 mg orally daily, and albuterol inhaler 2 puffs i.h. q4h-prn wheezing/soB. The patient's vital signs were within normal limits, and the patient was alert and interactive.

During the hospital stay, the patient's ascites improved significantly, and the patient was able to ambulate independently. The patient's mental status remained clear and coherent, and the patient was able to converse normally. The patient's neurological examination also remained intact, with no evidence of cerebellar or extrapy


In [33]:
response=query_engine.query("Summarize in 1-2 sentences why the patient admitted to the hospital. E.g. This patient with a history of MI, AAA, COPD, and hypertension was admitted to the hospital after presenting with chest pain.")
print(response)

The patient was admitted to the hospital after presenting with worsening abdominal distension and pain, which was likely caused by ascites due to cirrhosis and HCV infection. The patient's history of MEDICINE and IVDU may have contributed to the development of ascites, and her recent discontinuation of Lasix and spirnolactone may have exacerbated the condition.


In [34]:
response=query_engine.query("What were the treatments in the emergency room?")
print(response)

The treatments in the emergency room were:

1. Albuterol Inhaler 2 PUFF IH Q4H:PRN wheezing, SOB
2. Emtricitabine -Tenofovir (Truvada) 1 TAB PO DAILY
3. Furosemide 40 mg PO DAILY
4. Ipratropium Bromide Neb 1 NEB IH Q6H SOB
5. Nicotine Patch 14 mg TD DAILY

Note: The information provided is based on the context information given and should not be considered as medical advice.
